In [1]:

import torch
import torch.nn as nn
import torch.optim as optim

# Define a custom neural network class
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        
        # Define the network layers
        self.linear1 = nn.Linear(input_size, hidden_size) # First linear layer
        self.activation1 = nn.ReLU() # Activation function after the first layer
        self.linear2 = nn.Linear(hidden_size, output_size) # Second linear layer
        self.softmax = nn.Softmax(dim=1) # Softmax layer for the output
    
    def forward(self, x):
        # Define the forward pass
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

# Create an instance of the network
input_size = 10
hidden_size = 5
output_size = 3
model = SimpleNN(input_size, hidden_size, output_size)

# Create a loss function
criterion = nn.CrossEntropyLoss()

# Create an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Define a batch of inputs and targets
batch_size = 4
input_data = torch.randn(batch_size, input_size) # Random input tensor with batch size
targets = torch.randint(0, output_size, (batch_size,)) # Random target tensor for each input in the batch

# Forward pass
outputs = model(input_data)

# Compute loss
loss = criterion(outputs, targets)

# Backward pass
loss.backward()

# Update weights
optimizer.step()

# Optionally, print output and loss
print(f"Outputs:\n{outputs}")
print(f"Loss: {loss.item()}")


Outputs:
tensor([[0.3220, 0.1954, 0.4826],
        [0.2762, 0.2798, 0.4440],
        [0.2625, 0.2510, 0.4864],
        [0.2631, 0.2614, 0.4754]], grad_fn=<SoftmaxBackward0>)
Loss: 1.0052919387817383


In [7]:
# random array and save it to a file .npy
# import numpy as np

# B = 1000
# N = 100
# M = 30
# X = np.random.rand(B, N).astype(np.float32)
# W = np.random.rand(M,N).astype(np.float32)
# bias = np.random.rand(M).astype(np.float32)
# np.save('../with-torch-tests/linear-layer/X.npy', X)
# np.save('../with-torch-tests/linear-layer/W.npy', W)
# np.save('../with-torch-tests/linear-layer/bias.npy', bias)

# l = nn.Linear(N,M)
# l.weight.data = torch.from_numpy(W)
# l.bias.data = torch.from_numpy(bias)
# X_torch = torch.from_numpy(X)
# Y = l(X_torch)
# np.save('../with-torch-tests/linear-layer/Y.npy', Y.detach().numpy())
# print(Y[0,0:5])


tensor([27.6292, 23.5983, 28.8660, 25.8777, 24.1224], grad_fn=<SliceBackward0>)


In [32]:
# read arrays from .npy files and func Linear to compare
import numpy as np
import torch
import torch.nn as nn

X = np.load('../with-torch-tests/linear-layer/X_C.npy')
W = np.load('../with-torch-tests/linear-layer/W_C.npy')
bias = np.load('../with-torch-tests/linear-layer/bias_C.npy')
Y = np.load('../with-torch-tests/linear-layer/out_C.npy')
B,N = X.shape
_,M = Y.shape
l = nn.Linear(M,N)
l.weight.data = torch.from_numpy(W).to(torch.float32)
l.bias.data = torch.from_numpy(bias).to(torch.float32)
X_torch = torch.from_numpy(X).to(torch.float32)
Y_torch = l(X_torch)

print(Y[0,0:5])
print(Y_torch[0,0:5])
print( np.allclose(Y, Y_torch.detach().numpy(), atol=1e-4, rtol=1e-4))



[-3.7956476  -3.9812362   0.08876852  1.2680509   0.22692043]
tensor([-3.7956, -3.9812,  0.0888,  1.2681,  0.2269], grad_fn=<SliceBackward0>)
True


In [39]:
# Relu forward and backward tests
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load data
X = np.load(r'../with-torch-tests/relu-layer/X_relu.npy')
Y = np.load(r'../with-torch-tests/relu-layer/out_relu.npy')
dY = np.load(r'../with-torch-tests/relu-layer/up_grad_relu.npy')
dX = np.load(r'../with-torch-tests/relu-layer/down_grad_relu.npy')
# Convert to PyTorch tensors
# make sure to set requires_grad=True for the input tensor so that the Autograd engine can compute the gradients
X_torch = torch.from_numpy(X).to(torch.float32).requires_grad_(True)
dY_torch = torch.from_numpy(dY).to(torch.float32)

# Forward pass with ReLU
relu = nn.ReLU()
Y_torch = relu(X_torch)

# Compare the forward pass results
print("Forward pass comparison:")
print("Y (numpy):", Y[0, 0:5])
print("Y_torch:", Y_torch.detach().numpy()[0, 0:5])
print("Match:", np.allclose(Y, Y_torch.detach().numpy(), atol=1e-4, rtol=1e-4))


# Validate the backward pass
Y_torch.backward(dY_torch)

# Get the gradients from X_torch
dX_torch = X_torch.grad

# Compare the backward pass results
print("\nBackward pass comparison:")
print("dX (numpy):", dX[0, 0:5])
print("dX_torch:", dX_torch.numpy()[0, 0:5])
print("Match:", np.allclose(dX, dX_torch.numpy(), atol=1e-4, rtol=1e-4))


Forward pass comparison:
Y (numpy): [0.09152496 0.         0.3774835  0.         0.59312725]
Y_torch: [0.09152496 0.         0.3774835  0.         0.59312725]
Match: True

Backward pass comparison:
dX (numpy): [-0.9976806   0.          0.29630423  0.         -0.45951718]
dX_torch: [-0.9976806   0.          0.29630423  0.         -0.45951718]
Match: True
